In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Concatenate, Reshape
from tensorflow.keras import Input
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('../Data/train.csv')
df_train['binary_open_channels'] = [int(x != 0) for x in df_train.open_channels]
df_train

,time,signal,open_channels,binary_open_channels
0,0.0001,-2.7600,0,0
1,0.0002,-2.8557,0,0
2,0.0003,-2.4074,0,0
3,0.0004,-3.1404,0,0
4,0.0005,-3.1525,0,0
...,...,...,...,...
4999995,499.9996,2.9194,7,1
4999996,499.9997,2.6980,7,1
4999997,499.9998,4.5164,8,1
4999998,499.9999,5.6397,9,1


In [3]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train.open_channels)

max_val = max(signal)
min_val = min(signal)
diff = max_val-min_val
signal = (signal - min_val)/diff

data = np.zeros((len(signal), 2))
data[:, 0] = signal
data[:, 1] = open_channels
data = data.reshape((1000, 5000, 2))
np.random.shuffle(data)

x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1].reshape((1000,5000,1))

In [4]:
np.shape(x),np.shape(y)

((1000, 5000, 1), (1000, 5000, 1))

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .3)

In [6]:
encoding_input = Input(shape=(5000,1,))

reshape_1 = Reshape(target_shape=(5000,))(encoding_input)
encoding = Dense(100)(reshape_1)
decoding = Dense(5000)(encoding)

encoding_output = Reshape(target_shape=(5000,1,))(decoding)

In [7]:
encoder = Model(encoding_input,encoding_output)
encoder.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])
encoder.summary()
encoder.fit(x_train, x_train, epochs=100, validation_data=(x_test,x_test),verbose=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5000, 1)]         0         
_________________________________________________________________
reshape (Reshape)            (None, 5000)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               500100    
_________________________________________________________________
dense_1 (Dense)              (None, 5000)              505000    
_________________________________________________________________
reshape_1 (Reshape)          (None, 5000, 1)           0         
Total params: 1,005,100
Trainable params: 1,005,100
Non-trainable params: 0
_________________________________________________________________
Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 2s 2ms/sample - 

In [8]:
x_train_encoded = encoder.predict(x_train)
x_test_encoded = encoder.predict(x_test)

In [6]:
inputs = Input(shape=(5000,1,))

# reshape_1 = Reshape(target_shape=(5000,))(inputs)
# encoding = Dense(1000)(reshape_1)
# decoding = Dense(5000)(encoding)
# reshape_2 = Reshape(target_shape=(5000,1,))(decoding)

layer1_a = LSTM(units=10, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_a = Dense(16,activation='relu')(layer1_a)

layer1_b = SimpleRNN(units=10, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_b = Dense(16,activation='relu')(layer1_b)
concat = Concatenate()([layer2_a, layer2_b])

outputs = Dense(units=1, activation='relu')(concat)

In [7]:
model = Model(inputs,outputs)
model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5000, 1)]    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 5000, 10)     480         input_1[0][0]                    
__________________________________________________________________________________________________
simple_rnn (SimpleRNN)          (None, 5000, 10)     120         input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 5000, 16)     176         lstm[0][0]                       
______________________________________________________________________________________________

In [8]:
model.fit(x_train, y_train, epochs=25, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/25
700/700 [==============================] - 170s 243ms/sample - loss: 10.5055 - accuracy: 0.2211 - val_loss: 7.3358 - val_accuracy: 0.1964
Epoch 2/25
700/700 [==============================] - 169s 241ms/sample - loss: 6.1025 - accuracy: 0.1975 - val_loss: 6.3447 - val_accuracy: 0.1964
Epoch 3/25
700/700 [==============================] - 169s 242ms/sample - loss: 5.4910 - accuracy: 0.1975 - val_loss: 5.4669 - val_accuracy: 0.1964
Epoch 4/25
700/700 [==============================] - 172s 245ms/sample - loss: 4.2743 - accuracy: 0.1975 - val_loss: 3.4031 - val_accuracy: 0.1964
Epoch 5/25
700/700 [==============================] - 174s 248ms/sample - loss: 3.2062 - accuracy: 0.2043 - val_loss: 3.1917 - val_accuracy: 0.1964
Epoch 6/25
700/700 [==============================] - 170s 243ms/sample - loss: 2.9361 - accuracy: 0.2644 - val_loss: 3.1775 - val_accuracy: 0.1964
Epoch 7/25
700/700 [==============================] - 165s 236ms/